In [1]:
import pandas as pd

Load the NBA Salary dataset from 1990 to 2017

In [2]:
## Data Source: https://data.world/datadavis/nba-salaries
salary = pd.read_excel('../data/raw/NBA_Salary_1990-2017.xlsx')
salary = salary.drop(columns = 'Register Value')
salary.head()

,Player Name,Salary in $,Season Start,Season End,Team,Full Team Name
0,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns


Load the NBA Salary dataset from 2018 to 2019 and clean the headers so that it matches the format from 1990 to 2017

In [3]:
salary201819 = pd.read_csv('../data/raw/NBA_Salary_2018-2019.csv')
salary201819['Player Name'] = salary201819['Player'].str.split(pat = '\\', expand = True)[0]
salary201819['Salary in $'] = salary201819['2018-19'].str.slice(start = 1)
salary201819 = salary201819[['Player Name', 'Tm', 'Salary in $']].rename(columns = {'Tm': 'Team'})
salary201819['Season Start'] = 2018
salary201819['Season End'] = 2019
salary201819 = pd.merge(salary201819, salary[['Team', 'Full Team Name']].drop_duplicates(), on = 'Team', how = 'left')
salary201819.head()

,Player Name,Team,Salary in $,Season Start,Season End,Full Team Name
0,Stephen Curry,GSW,37457154,2018,2019,Golden State Warriors
1,Chris Paul,HOU,35654150,2018,2019,Houston Rockets
2,Russell Westbrook,OKC,35654150,2018,2019,Oklahoma City Thunder
3,LeBron James,LAL,35654150,2018,2019,Los Angeles Lakers
4,Blake Griffin,DET,32088932,2018,2019,Detroit Pistons


Add the 2018 - 2019 data to 1990 - 2017

In [4]:
salarydf = pd.concat([salary, salary201819], sort = True)
salarydf.head()

,Full Team Name,Player Name,Salary in $,Season End,Season Start,Team
0,Los Angeles Lakers,A.C. Green,1750000,1991,1990,LAL
1,Los Angeles Lakers,A.C. Green,1750000,1992,1991,LAL
2,Los Angeles Lakers,A.C. Green,1750000,1993,1992,LAL
3,Phoenix Suns,A.C. Green,1885000,1994,1993,PHO
4,Phoenix Suns,A.C. Green,6472600,1995,1994,PHO


Load the NBA Team Abbreviations dataset

In [5]:
abb = pd.read_csv('../data/scraped/NBA_Teams_Abb.csv')
abb.head()

,Abbreviation/Acronym,Franchise
0,ATL,Atlanta Hawks
1,BKN,Brooklyn Nets
2,BOS,Boston Celtics
3,CHA,Charlotte Hornets
4,CHI,Chicago Bulls


Join the Salary DataFrame and Abbreviations dataset

In [6]:
salarydf_abb = pd.merge(salarydf, abb, left_on = 'Team', right_on = 'Abbreviation/Acronym', how = 'left')
salarydf_abb.head()

,Full Team Name,Player Name,Salary in $,Season End,Season Start,Team,Abbreviation/Acronym,Franchise
0,Los Angeles Lakers,A.C. Green,1750000,1991,1990,LAL,LAL,Los Angeles Lakers
1,Los Angeles Lakers,A.C. Green,1750000,1992,1991,LAL,LAL,Los Angeles Lakers
2,Los Angeles Lakers,A.C. Green,1750000,1993,1992,LAL,LAL,Los Angeles Lakers
3,Phoenix Suns,A.C. Green,1885000,1994,1993,PHO,NaN,NaN
4,Phoenix Suns,A.C. Green,6472600,1995,1994,PHO,NaN,NaN


##### Find the misspelled abbreviation team names

In [7]:
misabb = salarydf_abb.loc[salarydf_abb['Abbreviation/Acronym'].isna(), ['Team', 'Full Team Name']].drop_duplicates()
misabb = pd.merge(misabb, abb, left_on = 'Full Team Name', right_on = 'Franchise', how = 'left')
misabb

,Team,Full Team Name,Abbreviation/Acronym,Franchise
0,PHO,Phoenix Suns,PHX,Phoenix Suns
1,NOH,New Orleans Pelicans,NOP,New Orleans Pelicans
2,NJN,Brooklyn Nets,BKN,Brooklyn Nets
3,CHO,NaN,NaN,NaN
4,BRK,NaN,NaN,NaN


##### After searching online, we know that the true meaning of 'CHO' is 'Charlotte Hornets'and 'BRK' is 'Brooklyn Nets'

In [8]:
abb.loc[(abb.Franchise == 'Brooklyn Nets') | (abb.Franchise == 'Charlotte Hornets')]

,Abbreviation/Acronym,Franchise
1,BKN,Brooklyn Nets
3,CHA,Charlotte Hornets


##### Change Abb Team Name with a dictionary

In [9]:
change_abb_dic = {'PHO': 'PHX', 'NOH': 'NOP', 'NJN': 'BKN', 'CHO': 'CHA', 'BRK': 'BKN'}
salarydf['Team'] = salarydf["Team"].replace(change_abb_dic)

##### Left join again to find the true full team name

In [10]:
salary_new_df = pd.merge(salarydf, abb, left_on = 'Team', right_on = 'Abbreviation/Acronym', how = 'left')
salary_new_df = salary_new_df.drop(columns = ['Full Team Name', 'Abbreviation/Acronym'])
salary_new_df.head()

,Player Name,Salary in $,Season End,Season Start,Team,Franchise
0,A.C. Green,1750000,1991,1990,LAL,Los Angeles Lakers
1,A.C. Green,1750000,1992,1991,LAL,Los Angeles Lakers
2,A.C. Green,1750000,1993,1992,LAL,Los Angeles Lakers
3,A.C. Green,1885000,1994,1993,PHX,Phoenix Suns
4,A.C. Green,6472600,1995,1994,PHX,Phoenix Suns


##### Save the df to csv

In [11]:
salary_new_df.to_csv('../data/cleaned/NBA_Salary.csv', index = False)